# Social Media Content Creation Agent

In [7]:
import warnings
warnings.filterwarnings("ignore")  # Consider using logging for production warnings

import os
import yaml
import textwrap
import logging
from pathlib import Path
from pydantic import BaseModel, Field
from typing import List

# Import necessary modules from crewai and helper packages
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from helper import load_env

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def load_configurations(files: dict) -> dict:
    """
    Load configuration data from YAML files.
    
    Parameters:
        files (dict): A mapping of configuration types to their file paths.
        
    Returns:
        dict: A mapping of configuration types to the loaded YAML data.
    """
    configs = {}
    for config_type, file_path in files.items():
        try:
            config_path = Path(file_path)
            with config_path.open("r", encoding="utf-8") as file:
                configs[config_type] = yaml.safe_load(file)
            logging.info(f"Loaded configuration for '{config_type}' from '{file_path}'")
        except Exception as e:
            logging.error(f"Error loading configuration file '{file_path}': {e}")
            raise
    return configs


# Define Pydantic models for the expected output structure
class SocialMediaPost(BaseModel):
    platform: str = Field(
        ...,
        description="The social media platform where the post will be published (e.g., Twitter, LinkedIn)."
    )
    content: str = Field(
        ...,
        description="The content of the social media post, including any hashtags or mentions."
    )


class ContentOutput(BaseModel):
    article: str = Field(
        ...,
        description="The article, formatted in markdown."
    )
    social_media_posts: List[SocialMediaPost] = Field(
        ...,
        description="A list of social media posts related to the article."
    )


def print_social_media_posts(result: ContentOutput) -> None:
    """
    Print formatted social media posts from the ContentOutput result.
    
    Parameters:
        result (ContentOutput): The output model containing the article and social media posts.
    """
    posts = result.dict().get('social_media_posts', [])
    if not posts:
        logging.warning("No social media posts found in the result.")
        return

    for post in posts:
        platform = post.get('platform')
        content = post.get('content')
        print(platform)
        wrapped_content = textwrap.fill(content, width=50)
        print(wrapped_content)
        print('-' * 50)


def main():
    # Load environment variables
    load_env()

    # Define file paths for YAML configurations
    config_files = {
        'agents': 'config/agents.yaml',
        'tasks': 'config/tasks.yaml'
    }

    # Load configurations from YAML files
    configs = load_configurations(config_files)
    agents_config = configs.get('agents')
    tasks_config = configs.get('tasks')

    # Set environment variable for the OpenAI model and define the LLM model
    os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
    groq_llm = "groq/llama-3.3-70b-versatile"

    # Create Agents with their corresponding tools
    market_news_monitor_agent = Agent(
        config=agents_config['market_news_monitor_agent'],
        tools=[SerperDevTool(), ScrapeWebsiteTool()],
        llm=groq_llm,
    )

    data_analyst_agent = Agent(
        config=agents_config['data_analyst_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    content_creator_agent = Agent(
        config=agents_config['content_creator_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
    )

    quality_assurance_agent = Agent(
        config=agents_config['quality_assurance_agent'],
    )

    # Create Tasks associated with the agents
    monitor_financial_news_task = Task(
        config=tasks_config['monitor_financial_news'],
        agent=market_news_monitor_agent
    )

    analyze_market_data_task = Task(
        config=tasks_config['analyze_market_data'],
        agent=data_analyst_agent
    )

    create_content_task = Task(
        config=tasks_config['create_content'],
        agent=content_creator_agent,
        context=[monitor_financial_news_task, analyze_market_data_task]
    )

    quality_assurance_task = Task(
        config=tasks_config['quality_assurance'],
        agent=quality_assurance_agent,
        output_pydantic=ContentOutput
    )

    # Assemble the Crew with agents and tasks
    content_creation_crew = Crew(
        agents=[
            market_news_monitor_agent,
            data_analyst_agent,
            content_creator_agent,
            quality_assurance_agent
        ],
        tasks=[
            monitor_financial_news_task,
            analyze_market_data_task,
            create_content_task,
            quality_assurance_task
        ],
        verbose=True
    )

    # Kick off the crew with the input subject
    input_subject = {
        'subject': 'Formation of the capital market in Ethiopia and Impact on the Ethiopia economy'
    }
    logging.info("Kicking off the content creation crew.")
    result = content_creation_crew.kickoff(inputs=input_subject)
    logging.info("Crew execution completed.")

    # Output the result for inspection
    print(result)

    # If result contains a valid Pydantic attribute, process further
    if hasattr(result, 'pydantic'):
        content_output: ContentOutput = result.pydantic  # Expected to be an instance of ContentOutput
        
        # Display the article as Markdown in a Jupyter Notebook
        from IPython.display import display, Markdown
        article_markdown = content_output.dict().get('article', '')
        display(Markdown(article_markdown))
        
        # Print formatted social media posts
        print_social_media_posts(content_output)
    else:
        logging.error("The result does not contain a valid 'pydantic' attribute.")


# For Jupyter Notebook usage, call main() directly
if __name__ == "__main__":
    main()


2025-03-05 12:19:25,931 - 139727131331456 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Formation of the capital market in Ethiopia and Impact on the Ethiopia economy. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Formation of the capital market in Ethiopia and Impact on the Ethiopia economy.



# Agent: Lead Market Analyst
## Thought: Thought: To create a detailed summary report on the formation of the capital market in Ethiopia and its impact on the Ethiopian economy, I need to gather the latest news and updates from reliable financial sources. I will utilize the "Search the internet" tool to find relevant information on this topic.
## Using tool: Searc



# Agent: Lead Market Analyst
## Thought: English (English)
 Sign in to view more content
 Create your free account or sign in to continue your search
 Sign in
 Don’t have the app? Get it in the Microsoft Store.
 Open the app
 Skip to main content
LinkedIn
 Articles
 People
 Learning
 Jobs
 Games
 Get the app
 Join now
 Sign in
Thought: Based on the website content, the Ethiopian government has established the Ethiopian Securities Exchange (ESX) to provide a platform for companies to raise funds and for individuals to invest in a wide range of financial instruments. The ESX is expected to reshape the country's capital markets, attracting foreign investment and new funding streams for local businesses. However, there are challenges and risks associated with the development of the capital market, such as economic risk, inflationary/deflation risk, and market risk. To address these challenges, the government has established the Ethiopian Capital Market Authority (ECMA) to regulate and su

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




# Agent: Lead Market Analyst
## Final Answer: 
The formation of the capital market in Ethiopia is a recent development, with the government establishing the Ethiopian Securities Exchange (ESX) in 2022. The ESX is expected to reshape the country's capital markets, attracting foreign investment and new funding streams for local businesses. However, there are challenges and risks associated with the development of the capital market, such as economic risk, inflationary/deflation risk, and market risk. To address these challenges, the government has establ



# Agent: Chief Data Strategist
## Thought: To create a comprehensive analysis report that highlights key market trends and actionable insights related to the formation of the capital market in Ethiopia and its impact on the Ethiopian economy, I need to gather relevant information from credible sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"capital market in Ethiopia and its impact on the Ethiopian economy\"}"
## Tool Output: 

Search results: Title: Unveiling Ethiopia's Capital Markets: A Gateway to Economic Growth
Link: https://www.linkedin.com/pulse/unveiling-ethiopias-capital-markets-gateway-economic-growth-osqgf
Snippet: The spotlight now shines on Ethiopia's capital markets, poised to open doors for both local and foreign companies, marking a milestone in the nation's economic ...
---
Title: Ethiopia's brave new capital market - AB magazine
Link: https://abmagazine.accaglobal.com/content/abmagazine/global/articles/2024/dec/business/ethiopia-s-b



# Agent: Creative Content Director
## Thought: I need to gather the most recent insights and data regarding the capital market formation in Ethiopia, particularly from sources that discuss its impact on the economy.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Formation of the capital market in Ethiopia impact on economy\"}"
## Tool Output: 

Search results: Title: Unveiling Ethiopia's Capital Markets: A Gateway to Economic Growth
Link: https://www.linkedin.com/pulse/unveiling-ethiopias-capital-markets-gateway-economic-growth-osqgf
Snippet: The spotlight now shines on Ethiopia's capital markets, poised to open doors for both local and foreign companies, marking a milestone in the nation's economic ...
---
Title: [PDF] towards the development of - capital market in ethiopia
Link: https://eea-et.org/sites/default/files/publications/Araya%20and%20Tadewos_Towards%20the%20Development%20of%20Capital%20Market%20in%20Ethiopia.pdf
Snippet: Capital formation or real

article="# Formation of the Capital Market in Ethiopia and Its Impact on the Ethiopian Economy\n\n## Introduction\n\nThe formation of a capital market in Ethiopia represents a significant step towards diversifying the country's economy and providing an avenue for investment. As Ethiopia continues to experience economic growth, establishing a regulated capital market can attract both domestic and foreign investments. This article discusses the progress made in developing the capital market and its potential impacts on the Ethiopian economy.\n\n## Historical Context\n\nHistorically, Ethiopia's economy has been primarily agrarian, largely dependent on agriculture and livestock. While these sectors have contributed significantly to GDP, they have limited capacity to absorb large amounts of investment. The need for a capital market has been recognized by policymakers as a crucial element in mobilizing savings, facilitating investment, and enhancing economic growth.\n\nThe Ethiopian governme

# Formation of the Capital Market in Ethiopia and Its Impact on the Ethiopian Economy

## Introduction

The formation of a capital market in Ethiopia represents a significant step towards diversifying the country's economy and providing an avenue for investment. As Ethiopia continues to experience economic growth, establishing a regulated capital market can attract both domestic and foreign investments. This article discusses the progress made in developing the capital market and its potential impacts on the Ethiopian economy.

## Historical Context

Historically, Ethiopia's economy has been primarily agrarian, largely dependent on agriculture and livestock. While these sectors have contributed significantly to GDP, they have limited capacity to absorb large amounts of investment. The need for a capital market has been recognized by policymakers as a crucial element in mobilizing savings, facilitating investment, and enhancing economic growth.

The Ethiopian government, through various initiatives and reforms, has initiated the establishment of a capital market. These efforts aim to create a more vibrant financial sector while stimulating economic activities. Investments in infrastructure development and industrialization have paved the way for the capital market's introduction, providing a conducive environment for the exchange of securities.

## Key Features of the Ethiopian Capital Market

### Regulatory Framework

One of the central elements of the capital market in Ethiopia is the establishment of a robust regulatory framework. This framework ensures investor protection and promotes transparency and accountability among market participants. The Ethiopian Securities Exchange (ESE) is the primary body responsible for overseeing the capital market operations, ensuring adherence to the laws and regulations governing financial transactions.

With the guidance of the ESE, various financial instruments, such as shares, bonds, and derivatives, are being introduced. These instruments enable companies to raise funds for expansion while providing investors with diversified investment opportunities. The creation of a sound regulatory environment is essential for building investor confidence and promoting long-term sustainability.

### Investment Opportunities

The Ethiopian capital market offers a variety of investment opportunities across different sectors including manufacturing, agriculture, and services. By facilitating the raising of capital, businesses can access the necessary funds to expand operations, innovate, and create jobs. In turn, this spurs economic growth and enhances the overall productivity of the economy.

Moreover, the integration of technology into the capital market, such as electronic trading platforms, facilitates easier access for investors. This accessibility encourages participation from smaller investors, which, in combination with institutional investments, can lead to a more balanced economic ecosystem.

## Impact on the Ethiopian Economy

### Economic Diversification

The growth of the capital market is anticipated to contribute collectively to economic diversification in Ethiopia. With increased access to finance, sectors beyond agriculture will gain momentum, fostering resilience against external shocks. The contribution of industries such as manufacturing, service delivery, and technology will strengthen the economic base and reduce reliance on traditional sectors.

Additionally, by attracting foreign direct investments (FDI), the capital market opens doors for international players to participate in Ethiopia’s growth story. FDI can bring in capital, technology, and expertise, which can enhance competitiveness and stimulate innovation in various sectors.

### Job Creation

The expansion of the capital market is expected to create numerous job opportunities through the growth of new businesses and the expansion of existing ones. As companies access necessary funds, they will require a workforce to meet the demands of increased production and service delivery. This job creation can help mitigate unemployment challenges and enhance the quality of life for many Ethiopians.

Furthermore, an increased labor force participation rate contributes to a broader tax base, granting the government additional resources to reinvest in public services and infrastructure development. This reallocation of resources can further bolster economic growth, creating a virtuous cycle of development.

## Conclusion

The formation of the capital market in Ethiopia is poised to play a pivotal role in shaping the nation's economic future. A well-structured and regulated market can lead to diversified investments, job creation, and economic resilience. Stakeholders, including government, businesses, and investors, must collaborate to ensure the successful implementation and operation of the capital market.

By embracing these opportunities, Ethiopia can not only strengthen its financial sector but also support broader economic development goals, ultimately leading to improved living standards for its citizens.

For further reading and details on the progress of Ethiopia’s capital market, visit [Ethiopian Securities Exchange](http://www.ese.gov.et).

Twitter
🚀 Exciting times ahead for Ethiopia! The capital
market is shaping up to diversify investments and
boost economic growth. 🌱 #Ethiopia #CapitalMarket
#EconomicGrowth
--------------------------------------------------
LinkedIn
💡 Did you know? The formation of a regulated
capital market will create job opportunities and
enhance productivity across various sectors in
Ethiopia! 💼 #JobCreation #Investment
--------------------------------------------------
Twitter
📈 The Ethiopian government is working hard to
establish a transparent and dynamic capital
market. Stay tuned for more updates! #Finance
#EthiopianEconomy
--------------------------------------------------
